In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import time
from multiprocessing import Pool
import random
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# 保持原有的数据处理函数不变
def process_window(args):
    i, features_values, labels_values, WINDOW_SIZE, label_index = args
    window_features = features_values[i:i+WINDOW_SIZE]
    label_value = labels_values[i + WINDOW_SIZE + [5, 10, 20, 40, 60][label_index] - 1, label_index]
    return (window_features, label_value)

def process_single_file(file_path, label_index=0):
    data = pd.read_csv(file_path)
    data = data.sort_values('time')
    
    WINDOW_SIZE = 100
    PREDICTION_STEPS = [5, 10, 20, 40, 60]
    
    data['time'] = data['time'].astype('datetime64[ns]').dt.hour
    features = data.drop(columns=['date'])
    labels = data[['label_5', 'label_10', 'label_20', 'label_40', 'label_60']]
    
    scaler = StandardScaler()
    features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
    
    total_samples = len(data) - WINDOW_SIZE - PREDICTION_STEPS[label_index]
    
    features_values = features.values
    labels_values = labels.values
    
    args_list = [(i, features_values, labels_values, WINDOW_SIZE, label_index) for i in range(total_samples)]
    
    with Pool() as p:
        results = p.map(process_window, args_list)
    X, y = zip(*results)
    
    X = np.array(X)
    y = np.array(y)
    
    return train_test_split(X, y, test_size=0.3, shuffle=False, random_state=42)

def process_folder(folder_path, label_index=0, sample_size=None, random_seed=42):
    all_X_train = []
    all_y_train = []
    all_X_test = []
    all_y_test = []
    
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    random.seed(random_seed)
    if sample_size is not None and sample_size < len(csv_files):
        csv_files = random.sample(csv_files, sample_size)
    
    print(f"Processing {len(csv_files)} files: {csv_files}")
    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        X_train, X_test, y_train, y_test = process_single_file(file_path, label_index)
        
        all_X_train.append(X_train)
        all_y_train.append(y_train)
        all_X_test.append(X_test)
        all_y_test.append(y_test)
    
    final_X_train = np.concatenate(all_X_train, axis=0)
    final_y_train = np.concatenate(all_y_train, axis=0)
    final_X_test = np.concatenate(all_X_test, axis=0)
    final_y_test = np.concatenate(all_y_test, axis=0)
    
    return final_X_train, final_X_test, final_y_train, final_y_test

# 加载数据
label_index = 0 
folder_path = 'train_set'
sample_size = 50

X_train, X_test, y_train, y_test = process_folder(
    folder_path, 
    label_index=label_index, 
    sample_size=sample_size
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# 训练LightGBM模型并获取预测概率作为特征
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# LightGBM参数
params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'seed': 42
}

num_round = 200
train_data = lgb.Dataset(X_train_reshaped, label=y_train)
test_data = lgb.Dataset(X_test_reshaped, label=y_test, reference=train_data)

bst = lgb.train(params, 
               train_data, 
               num_round, 
               valid_sets=[test_data], 
               callbacks=[lgb.early_stopping(stopping_rounds=10)])

# 获取LightGBM的预测概率
lgb_train_probs = bst.predict(X_train_reshaped)
lgb_test_probs = bst.predict(X_test_reshaped)

# 修改LSTM模型以接受LightGBM特征
class HybridLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, lgb_feature_dim=3, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, num_layers,
            batch_first=True, bidirectional=True
        )
        self.dropout = nn.Dropout(dropout)
        self.attention = nn.Sequential( 
            nn.Linear(hidden_dim*2, 1),
            nn.Softmax(dim=1)
        )
        # 增加一个全连接层来处理LSTM输出
        self.lstm_fc = nn.Linear(hidden_dim*2, hidden_dim)
        # 处理LightGBM特征的层
        self.lgb_fc = nn.Linear(lgb_feature_dim, hidden_dim)
        # 最终分类层
        self.final_fc = nn.Linear(hidden_dim*2, num_classes)
    
    def forward(self, x, lgb_features):
        lstm_out, _ = self.lstm(x)
        
        attn_weights = self.attention(lstm_out)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        
        # LSTM分支
        lstm_features = self.lstm_fc(context)
        lstm_features = self.dropout(lstm_features)
        
        # LightGBM分支
        lgb_features = self.lgb_fc(lgb_features)
        
        # 合并特征
        combined = torch.cat([lstm_features, lgb_features], dim=1)
        
        return self.final_fc(combined)

# 设置设备
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 准备数据
input_dim = 33
hidden_dim = 32
num_layers = 2
num_epochs = 50
num_classes = 3

# 将LightGBM概率特征转换为tensor
lgb_train_tensor = torch.from_numpy(lgb_train_probs).float().to(device)
lgb_test_tensor = torch.from_numpy(lgb_test_probs).float().to(device)

# 原始特征tensor
X_train_tensor = torch.from_numpy(X_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).long().to(device)
X_test_tensor = torch.from_numpy(X_test).float().to(device)
y_test_tensor = torch.from_numpy(y_test).long().to(device)

# 创建混合模型
model = HybridLSTM(input_dim, hidden_dim, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.RAdam(model.parameters(), lr=0.001)

print(model)

# 创建DataLoader
batch_size = 256
# 使用TensorDataset来组合原始特征和LightGBM特征
train_dataset = TensorDataset(X_train_tensor, lgb_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 训练循环
hist = np.zeros(num_epochs)
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for batch_x, batch_lgb, batch_y in train_loader:
        optimiser.zero_grad()
        outputs = model(batch_x, batch_lgb)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimiser.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    hist[epoch] = avg_loss
    print(f"Epoch {epoch}, Avg Loss: {avg_loss}")

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

# 评估模型
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor, lgb_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f"Test Accuracy: {accuracy:.4f}")

# 保存模型
torch.save(model.state_dict(), f"hybrid_model_label_{[5,10,20,40,60][label_index]}.pth")
print(f"Model saved as hybrid_model_label_{[5,10,20,40,60][label_index]}.pth")

Processing 50 files: ['snapshot_sym4_date58_pm.csv', 'snapshot_sym3_date30_pm.csv', 'snapshot_sym2_date50_am.csv', 'snapshot_sym2_date29_am.csv', 'snapshot_sym4_date16_am.csv', 'snapshot_sym0_date93_pm.csv', 'snapshot_sym3_date28_am.csv', 'snapshot_sym4_date42_am.csv', 'snapshot_sym1_date51_am.csv', 'snapshot_sym0_date44_pm.csv', 'snapshot_sym0_date96_pm.csv', 'snapshot_sym0_date76_am.csv', 'snapshot_sym3_date10_pm.csv', 'snapshot_sym1_date65_pm.csv', 'snapshot_sym3_date53_am.csv', 'snapshot_sym3_date97_pm.csv', 'snapshot_sym0_date20_pm.csv', 'snapshot_sym1_date9_pm.csv', 'snapshot_sym0_date54_pm.csv', 'snapshot_sym2_date98_am.csv', 'snapshot_sym3_date31_pm.csv', 'snapshot_sym0_date70_am.csv', 'snapshot_sym2_date45_am.csv', 'snapshot_sym3_date17_pm.csv', 'snapshot_sym4_date10_pm.csv', 'snapshot_sym3_date64_pm.csv', 'snapshot_sym0_date10_am.csv', 'snapshot_sym4_date18_am.csv', 'snapshot_sym0_date16_am.csv', 'snapshot_sym4_date16_pm.csv', 'snapshot_sym1_date44_am.csv', 'snapshot_sym0_dat